# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [2]:
df_flights = pd.read_csv('Data/flightsV2.csv')

In [3]:
df_flights

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay
0,2018-06-23,AA,AA_CODESHARE,AA,2989,OO,N902EV,2989,15295,TOL,...,13930,ORD,"Chicago, IL",827,849,N,82,1,213,NaN
1,2019-04-30,UA,UA_CODESHARE,UA,5221,OO,N943SW,5221,11413,DRO,...,11292,DEN,"Denver, CO",1130,1244,N,74,1,250,7.0
2,2018-12-25,UA,UA_CODESHARE,UA,6166,YV,N87337,6166,11140,CRP,...,12266,IAH,"Houston, TX",535,637,N,62,1,201,-4.0
3,2018-11-28,DL,DL,DL,2905,DL,N966AT,2905,11433,DTW,...,11996,GSP,"Greer, SC",2140,2335,N,115,1,508,-33.0
4,2018-05-06,UA,UA_CODESHARE,UA,3705,YX,N726YX,3705,12953,LGA,...,13930,ORD,"Chicago, IL",1929,2112,N,163,1,733,-13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-08-11,AA,AA_CODESHARE,AA,4653,YX,N434YX,4653,14730,SDF,...,13303,MIA,"Miami, FL",700,935,N,155,1,910,-13.0
99996,2018-08-10,DL,DL_CODESHARE,DL,3378,9E,N272PQ,3378,10721,BOS,...,11193,CVG,"Cincinnati, OH",540,758,N,138,1,752,47.0
99997,2019-04-02,DL,DL_CODESHARE,DL,5062,9E,N925XJ,5062,11298,DFW,...,11193,CVG,"Cincinnati, OH",1550,1900,N,130,1,812,-20.0
99998,2019-09-21,UA,UA_CODESHARE,UA,3402,YX,N654RW,3402,11057,CLT,...,11618,EWR,"Newark, NJ",1625,1829,N,124,1,529,-11.0


In [4]:
df_flights.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'arr_delay'],
      dtype='object')

In [5]:
## Drop some columns we do not need
df_flights.drop(columns = ['branded_code_share', 'mkt_unique_carrier', 'mkt_carrier', 'mkt_carrier_fl_num', 'op_unique_carrier', 'op_carrier_fl_num', 'dup'], inplace = True)

In [8]:
df_flights.isnull().sum()

fl_date                 0
tail_num              316
origin_airport_id       0
origin                  0
origin_city_name        0
dest_airport_id         0
dest                    0
dest_city_name          0
crs_dep_time            0
crs_arr_time            0
crs_elapsed_time        0
flights                 0
distance                0
arr_delay            1960
dtype: int64

In [10]:
## Only a small fraction of data is missing so we will just drop those rows instead of trying to impute
df_flights.dropna(inplace = True)

In [11]:
df_flights

,fl_date,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay
1,2019-04-30,N943SW,11413,DRO,"Durango, CO",11292,DEN,"Denver, CO",1130,1244,74,1,250,7.0
2,2018-12-25,N87337,11140,CRP,"Corpus Christi, TX",12266,IAH,"Houston, TX",535,637,62,1,201,-4.0
3,2018-11-28,N966AT,11433,DTW,"Detroit, MI",11996,GSP,"Greer, SC",2140,2335,115,1,508,-33.0
4,2018-05-06,N726YX,12953,LGA,"New York, NY",13930,ORD,"Chicago, IL",1929,2112,163,1,733,-13.0
5,2018-11-14,N913EV,10397,ATL,"Atlanta, GA",10990,CHO,"Charlottesville, VA",1008,1145,97,1,457,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-08-11,N434YX,14730,SDF,"Louisville, KY",13303,MIA,"Miami, FL",700,935,155,1,910,-13.0
99996,2018-08-10,N272PQ,10721,BOS,"Boston, MA",11193,CVG,"Cincinnati, OH",540,758,138,1,752,47.0
99997,2019-04-02,N925XJ,11298,DFW,"Dallas/Fort Worth, TX",11193,CVG,"Cincinnati, OH",1550,1900,130,1,812,-20.0
99998,2019-09-21,N654RW,11057,CLT,"Charlotte, NC",11618,EWR,"Newark, NJ",1625,1829,124,1,529,-11.0


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [12]:
## Add Month and Day columns
df_flights['fl_date'] = pd.to_datetime(df_flights['fl_date'])

df_flights['month'] = df_flights['fl_date'].dt.month
df_flights['day'] = df_flights['fl_date'].dt.day

In [16]:
## Drop flights column
df_flights.drop(columns = 'flights', inplace = True)

In [18]:
df_flights.head(3)

,fl_date,tail_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,month,day
1,2019-04-30,N943SW,11413,DRO,"Durango, CO",11292,DEN,"Denver, CO",1130,1244,74,250,7.0,4,30
2,2018-12-25,N87337,11140,CRP,"Corpus Christi, TX",12266,IAH,"Houston, TX",535,637,62,201,-4.0,12,25
3,2018-11-28,N966AT,11433,DTW,"Detroit, MI",11996,GSP,"Greer, SC",2140,2335,115,508,-33.0,11,28


Lets Encode tail_num, origin airport and destination airport using average arrival delay for each of those airports

In [19]:
## Encode tail_num, origin airport and destination airport using mean arrival delay
df_tail = df_flights[['tail_num', 'arr_delay']]
df_origin = df_flights[['origin', 'arr_delay']]
df_dest = df_flights[['dest', 'arr_delay']]

In [23]:
df_tail = df_tail.groupby('tail_num', as_index = False).mean()
df_origin = df_origin.groupby('origin', as_index = False).mean()
df_dest = df_dest.groupby('dest', as_index = False).mean()

In [32]:
## Convert to dictionaries so we can use them to replace values 
dic_tail = df_tail.set_index('tail_num').to_dict()['arr_delay']
dic_origin = df_origin.set_index('origin').to_dict()['arr_delay']
dic_dest = df_dest.set_index('dest').to_dict()['arr_delay']

In [ ]:
## Create dataframe with features we want to use for modelling
df_model = df_flights[['tail_num', 'origin', 'dest', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance', 'arr_delay', 'month', 'day']]
df_model['tail_num'].replace(dic_tail, inplace = True)
df_model['origin'].replace(dic_origin, inplace = True)
df_model['dest'].replace(dic_dest, inplace = True)

In [39]:
df_model

,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,month,day
1,2.296296,2.854545,5.988561,1130,1244,74,250,7.0,4,30
2,20.875000,3.571429,5.687783,535,637,62,201,-4.0,12,25
3,-5.500000,5.122292,2.383673,2140,2335,115,508,-33.0,11,28
4,-3.954545,6.956799,10.413694,1929,2112,163,733,-13.0,5,6
5,5.250000,2.959515,1.326316,1008,1145,97,457,-15.0,11,14
...,...,...,...,...,...,...,...,...,...,...
99995,-3.952381,6.125364,7.095023,700,935,155,910,-13.0,8,11
99996,-0.608696,9.747001,9.022727,540,758,138,752,47.0,8,10
99997,-14.250000,10.153101,9.022727,1550,1900,130,812,-20.0,4,2
99998,-7.842105,6.280078,19.653990,1625,1829,124,529,-11.0,9,21


In [40]:
# df_model.to_csv('featuresV1.csv')
# df_flights.to_csv('flightsV3.csv')

We also want to add weather information

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.